# Final Research Project: Ammar Chalifah
**Based on Shidqie's Research**

In [9]:
import tensorflow as tf
from keras import layers
from tensorflow.keras import applications 
from keras.callbacks import Callback, ModelCheckpoint
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras import models

from sklearn.metrics import cohen_kappa_score, accuracy_score, confusion_matrix
from tqdm import tqdm

In [10]:
import numpy
import pandas

aptos_df = pandas.read_csv("../input/aptos2019-blindness-detection/train.csv")
aptos_dir = "../input/aptos2019-blindness-detection/train_images/"
images = []
labels = []

for idx in tqdm(range(len(aptos_df))):
#     images.append(load_ben_color(aptos_dir+aptos_df.iloc[idx]['id_code']+'.png', 20))
    labels.append(aptos_df.iloc[idx]['diagnosis'])
#     images.append(load_ben_color(dr_dir+dr_df.iloc[idx]['image']+'.jpeg'))
#     labels.append(dr_df.iloc[idx]['level'])

from tensorflow.keras.utils import to_categorical
images = numpy.load('../input/aptosnumpy/images.npy')
# images = numpy.array(images)
labels = numpy.array(labels)
# cat_labels = to_categorical(labels)

# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(images, cat_labels, 
#                                                     test_size=0.3, shuffle=True,
#                                                     random_state=42, stratify=labels)

100%|██████████| 3662/3662 [00:00<00:00, 9015.83it/s]


In [11]:
class Metrics(Callback):
    def __init__(self, xval, yval):
        super().__init__()
        self.xval = xval
        self.yval = yval
        
    def on_train_begin(self, logs={}):
        self.val_kappas = []

    def on_epoch_end(self, epoch, logs={}):
        X_val = self.xval
        y_val = self.yval
        
        y_pred = self.model.predict(X_val)
        y_pred = np.clip(y_pred,0,4)
        y_pred = y_pred.astype(int)

        _val_kappa = cohen_kappa_score(
            y_val,
            y_pred, 
            weights='quadratic'
        )

        self.val_kappas.append(_val_kappa)

        print(f"val_kappa: {_val_kappa:.4f}")
        
        if _val_kappa == max(self.val_kappas):
            print("Validation Kappa has improved. Saving model.")
            self.model.save('model.h5')

        return
    
kappa_metrics = Metrics(images, labels)

In [12]:
xception = applications.Xception(include_top=False, weights='imagenet')

83689472/83683744 [==============================] - 1s 0us/step


In [13]:
def build_model():
    model = Sequential()
    model.add(xception)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(256))
    model.add(layers.Dense(256))
    model.add(layers.Dense(1))
    
    model.compile(
        loss='mse',
        optimizer=Adam(lr=0.0001),
        metrics=['accuracy']
    )
    
    return model

In [14]:
model = build_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, None, None, 2048)  20861480  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               524544    
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 21,452,073
Trainable params: 21,397,545
Non-trainable params: 54,528
_________________________________________________________________


In [15]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
skf.get_n_splits(images, labels)

4

In [ ]:
for train_index, test_index in skf.split(images, labels):
    x_train, x_test = images[train_index], images[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    history = model.fit(x_train, y_train, epochs=8, batch_size=32,
    validation_data = (x_test, y_test), callbacks=[kappa_metrics])

Epoch 1/8
86/86 [==============================] - 52s 548ms/step - loss: 1.0165 - accuracy: 0.4819 - val_loss: 1.2528 - val_accuracy: 0.1015
val_kappa: 0.6697
Validation Kappa has improved. Saving model.
Epoch 2/8
86/86 [==============================] - 45s 524ms/step - loss: 0.3227 - accuracy: 0.5233 - val_loss: 0.3992 - val_accuracy: 0.5207
val_kappa: 0.9025
Validation Kappa has improved. Saving model.
Epoch 3/8
86/86 [==============================] - 45s 524ms/step - loss: 0.1857 - accuracy: 0.5701 - val_loss: 0.3853 - val_accuracy: 0.5229
val_kappa: 0.8901
Epoch 4/8
86/86 [==============================] - 45s 526ms/step - loss: 0.1285 - accuracy: 0.5660 - val_loss: 0.3571 - val_accuracy: 0.5568
val_kappa: 0.8671
Epoch 5/8
86/86 [==============================] - 45s 525ms/step - loss: 0.0955 - accuracy: 0.5847 - val_loss: 0.3575 - val_accuracy: 0.5469
val_kappa: 0.9112
Validation Kappa has improved. Saving model.
Epoch 6/8
86/86 [==============================] - 45s 524ms/step